<a href="https://colab.research.google.com/github/assafbz/HebrewNLP/blob/master/SpacyHebrewNer_Emnlp_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download EMNLP Dataset
Downloaded from https://sites.google.com/site/rmyeid/projects/polylgot-ner 
Al-Rfou, Rami for Polyglot Work

In [2]:
!pip install wget
import wget
url = 'http://cs.stonybrook.edu/~polyglot/ner2/emnlp_datasets.tgz'
wget.download(url)

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ad1e9fc79624275ffc0bdf7dd4a2158027a8ad977dac7aa20cfdfdff49eaea47
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


'emnlp_datasets.tgz'

# Extract Data
Only Hebrew Folder

In [3]:
!tar -xvf emnlp_datasets.tgz acl_datasets/he

acl_datasets/he/
acl_datasets/he/data/
acl_datasets/he/data/he_wiki.conll


# Prepare train data for SpaCy
note that text are rebuilt from tagged corpus cause no original texts were supplied

In [15]:
train_data = []
entities = []
sentence = ""
offset = 0
with open('acl_datasets/he/data/he_wiki.conll', 'r') as f:
    line = f.readline()
    while line:
      if line != '\n':
        token, tag = line.split('\t')
        tag = tag.rstrip()
        if tag != 'O':
          entity = (offset, offset + len(token), tag)
          entities.append(entity)
        sentence = sentence + token + ' '
        offset += len(token) + 1

      else:
        train_data.append((sentence, {"entities": entities}))
        entities = []
        sentence = ""
        offset = 0

      if len(train_data) > 10000:
        break;
      line = f.readline()
      
print("Retrieved", len(train_data), "Samples")

Retrieved 10001 Samples


# Check train data is correct and offsets are aligned

In [16]:
sample = train_data[2]
for entity in sample[1]["entities"]:
    print(sample[0][entity[0]:entity[1]], entity[2])

וורשה LOC


# Get Labels
Keep only distinct labels

In [17]:
all_labels = []
for doc in train_data:
    for entity in doc[1]['entities']:
        all_labels.append(entity[2])
labels = list(set(all_labels))
labels

['ORG', 'PER', 'LOC']

# Build Spacy Hebrew Ner Model
basic training

In [27]:
import spacy
from spacy.util import minibatch, compounding
from joblib import Parallel, delayed
from functools import partial
import random

spacy.require_gpu()
nlp = spacy.blank("he")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner, last=True)

random.shuffle(train_data)
for label in labels:
    ner.add_label(label)

optimizer = nlp.begin_training()  
batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
losses = {}
for batch in batches:
    texts, annotations = zip(*batch)
    nlp.update(
        texts,  # batch of texts
        annotations,  # batch of annotations
        drop=0.5,  # dropout - make it harder to memorise data
        losses=losses,
    )

Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Another Batch
Anothe

KeyboardInterrupt: ignored

# Test On Train Data Sample

In [25]:
test_text = train_data[1][0]
print(test_text)
print('==========================')
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

בהמשך המלחמה נפצע לוין ושבר את שתי רגליו במהלך פעילות היחידה ב שדה התעופה פאיד שב מצרים , אף על פי כן לאחר חצי שנה בלבד פיקד על כוח היחידה שכבש את שיא החרמון ‏ . 


# Test On New Sample

In [26]:
test_text = "כבר שנים רבות שמשרד הבריאות אינו מאפשר לאזרחי ישראל לבצע פעילות בסיסית"
print(test_text)
print('==========================')
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

כבר שנים רבות שמשרד הבריאות אינו מאפשר לאזרחי ישראל לבצע פעילות בסיסית
